# file path

In [71]:
INSERTS:str = "./../../testymolo/media/old_sql/VIRUS_1-2004-11-04_Inserts_2.sql"
TABLES: str = "./../../testymolo/media/old_sql/VIRUS_1-2004-11-04_Tables.sql"

tables_csv: str = "./../../testymolo/media/tables_csv/"


# tables and inserts

In [74]:
### caution when not all rows same nbr of col
def custom_csv_parser_to_list(key:str):
    infilepath: str = os.path.join(tables_csv, key+'.csv')
    with open(infilepath ,'r') as handle:

        rows:list = []

        for line in handle.readlines():
            line = line.strip(';') # removing terminal ';' 
            values:list = [] # re-initialize value list
            val:str = "" # current parsing value
            inquote:bool = False # re-initialize in quote : False
            quote_char:str = '' # ' or "
            is_string:bool = False
            for c in line:
                if(c == ','): # encounter comma
                    if(not inquote): ## and not in quote
                        val = val.strip("'")
                        val = val.strip('"')
                        if(len(val) > 0):
                            if(not is_string and val != "NULL" and val != "None"):
                                values.append(int(val))
                            else:
                                values.append(val)
                            is_string = False
                        else:
                            values.append(None)
                        val = ""
                        continue
                elif(c == '"' or c == "'"): # encounter quote
                    if(inquote): # already in quote 
                        if(quote_char == c):  # encounter ending quote mark
                            inquote = False
                        else: # encounter the other quote mark 
                            pass
                    else: # encounter starting quote mark
                        inquote = True
                        is_string = True
                        quote_char = c
                elif((c == ' ' or c == '\n') and not inquote):  # get rid of whitespaces
                    continue
                val += c
            if(len(val)>0):
                val = val.strip("'")
                val = val.strip('"')
                if(not is_string and val != "NULL" and val != "None"):
                    values.append(int(val))
                else:
                    values.append(val)
                is_string = False

            if(len(values) > 0):
                rows.append(values)
                #print(values)  
        
        #print("nbr of col", set([len(row) for row in rows]))
        #print("nbr of rows", len(rows))
        return rows

import os

data_keys:list = [ "Biblio_Struct", "CAZy_DB", "CAZy_GB_GP", "CAZy_PDB", "CAZy_PP", "CAZy_SP", "CAZyModO",
    "Cz_EC", "EC_num", "Fam_biblio", "ModO_Composition", "ModO_CrossRefs", "ModO_Families", "ModO_Limits",
    "Motifs", "Organism", "Prot_biblio", "Prot_Infos", "Prot_MOTIF", "Prot_MUT", "Prot_REG", "Prot_RI"]

data:dict = { key:custom_csv_parser_to_list(key) for key in data_keys }


In [81]:
### sort tables
def sort_tab(tab:list, primary_key_indexes:tuple=(0,1)):
    import operator
    new_tab:list = [   ]

    for j in range(len(tab)):
        for k in primary_key_indexes:
            tab[j][k] = int(tab[j][k])

    for i in reversed(primary_key_indexes):
        new_tab = sorted(tab, key=operator.itemgetter(i))
    return new_tab

"""
for tab in [ "CAZy_DB", "CAZy_GB_GP", "CAZy_PDB", "CAZy_SP",
    "Motifs", "Organism", "Cz_EC",]:
    data[tab] = sort_tab(data[tab], (0,))

for tab in ["CAZy_PP", "ModO_Composition", "ModO_Limits"]:
    data[tab] = sort_tab(data[tab], (0,1))

for tab in ["Prot_Infos", "Prot_MOTIF", "Prot_MUT", "Prot_REG", "Prot_RI"]:
    data[tab] = sort_tab(data[tab], (0,2))
"""

In [82]:
### write corrected tables

import csv

def write_table(key):
    infilepath: str = os.path.join(tables_csv, key+'.csv')
    with open(infilepath, 'w') as filehandler:
        for row in data[key]:
            line:str = ""
            for val in row:
                if(isinstance(val, str)):
                    line += "'"+val+"'"
                else:
                    line += str(val)
                line += ", "
            filehandler.write(line[:-2]+'\n')

#for tab in data:
#    write_table(tab)

## Exploration of results

In [83]:
## no assotiated : 609/2620
## multiple assotiated : 428/2620

import csv

no_associated_annotations:list = []
multiple_associated_annotations:list = []

with open("./Seq_annotation_list") as filehandler:
    csvcontent = csv.reader(filehandler)
    for row in csvcontent:
        if(row[-1] == "False"):
            no_associated_annotations.append(row)
        elif(len(row[-1].split(' ')) > 1):
            multiple_associated_annotations.append(row)

print(len(no_associated_annotations))
print(len(multiple_associated_annotations))


609
428


In [84]:
count:dict = {}
for row in no_associated_annotations:
    if(row[1] in count):
        count[row[1]] += 1
    else:
        count[row[1]] = 1

for item in count:
    print(f"{item}:{count[item]}/{len(data[item])}")

Prot_Infos:159/159
CAZy_PDB:68/68
CAZy_SP:3/4
Prot_MUT:9/9
Prot_RI:12/112
CAZy_PP:346/773
Prot_REG:12/42


## unfinished annotations per families

In [85]:
uncomplete_fams:list = []
with open("./Modulo_list") as filehandler:
    csvcontent = csv.reader(filehandler)
    for row in csvcontent:
        if(row[-1] == "False"):
            if(row[0] not in uncomplete_fams):
                uncomplete_fams.append(row[0])
print(uncomplete_fams)

['UNK', 'IHD', 'HD', 'ITM', 'TM', 'LNK', 'EX']
